# Librerías

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import date
from time import sleep
import pandas as pd
import os
import time

In [10]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
service = Service('/usr/bin/chromedriver')

In [11]:
op_menu_df = pd.read_excel('./facundo.xlsx',sheet_name='opciones_menu')
op_menu_df

,siberia,agrotecnia,salud,fceia,zaballa,casilda,centro
0,desayuno (7:45 - 10.30),NaN,desayuno (7:45 - 10.30),desayuno (7:45 - 10.30),NaN,NaN,desayuno (7:45 - 10.30)
1,llevar almuerzo (14 - 15),NaN,llevar almuerzo (14 - 15),llevar almuerzo (11 - 15),NaN,NaN,llevar almuerzo (14 - 15)
2,llevar almuerzo (13 - 14),NaN,llevar almuerzo (13 - 14),almuerzo (14 - 15),NaN,NaN,llevar almuerzo (13 - 14)
3,llevar almuerzo (12 - 13),NaN,llevar almuerzo (12 - 13),almuerzo (13 - 14),NaN,NaN,llevar almuerzo (12 - 13)
4,llevar almuerzo (11 - 12),NaN,llevar almuerzo (11 - 12),almuerzo (12 - 13),NaN,NaN,llevar almuerzo (11 - 12)
5,almuerzo (14 - 15),NaN,almuerzo (14 - 15),merienda (16 - 19),NaN,NaN,almuerzo (14 - 15)
6,almuerzo (13 - 14),NaN,almuerzo (13 - 14),llevar cena (20 - 21),NaN,NaN,almuerzo (13 - 14)
7,almuerzo (12 - 13),NaN,almuerzo (12 - 13),cena (20 - 21),NaN,NaN,almuerzo (12 - 13)
8,merienda (16 - 18),NaN,merienda (16 - 19),NaN,NaN,NaN,merienda (16 - 19)
9,llevar cena (19.30 - 21),NaN,llevar cena (19.30 - 21),NaN,NaN,NaN,llevar cena (19.30 - 21)


In [12]:
comedores_df = pd.read_excel('./facundo.xlsx',sheet_name='comedores')
comedores_df

,comedores
0,siberia
1,agrotecnia
2,salud
3,fceia
4,zaballa
5,casilda
6,centro


In [13]:
menues_elegidos_df = pd.read_excel('./facundo.xlsx',sheet_name='menues')
menues_elegidos_df.sort_values(by='fecha')

C:\Users\Usuario\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,comedor,menu,fecha
0,salud,desayuno (7:45 - 10.30),2025-06-02
1,salud,merienda (16 - 19),2025-06-02
2,salud,desayuno (7:45 - 10.30),2025-06-03
3,salud,merienda (16 - 19),2025-06-03
4,salud,desayuno (7:45 - 10.30),2025-06-04
5,siberia,merienda (16 - 18),2025-06-04
6,salud,desayuno (7:45 - 10.30),2025-06-05
7,salud,merienda (16 - 19),2025-06-05
8,salud,desayuno (7:45 - 10.30),2025-06-06
9,salud,merienda (16 - 19),2025-06-06


In [14]:
class Reserva:
    url_login = 'https://comedores.unr.edu.ar/'
    url_reservas = 'https://comedores.unr.edu.ar/comedor-reserva/reservar'
    comedor = None
    menu = None
    u_logueado = False
    # Ni en diciembre ni en enero se van a sacar turno para los meses siguientes
    meses = ['Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
    
    
    def __init__(self,u_dni:'str',u_clave:'str',op_eledigas:pd.DataFrame):
        self.driver = webdriver.Chrome(options=chrome_options)
        self.u_dni = u_dni
        self.u_clave = u_clave
        self.op_eledigas = op_eledigas
    def cerrar_navegador(self):
        print('se cierra el navegador\n')
        self.driver.quit()
    
    def loguearse(self):
        # Abrir la página web
        try:
            self.driver.get(self.url_login)
            # Capturar el elemento formulario
            e_form = WebDriverWait(driver=self.driver, timeout=2).until(
                EC.presence_of_element_located(locator=(By.ID, "form-login"))
            )
            print('Formulario de logeo recibido')
            # Capturar los elementos del formulario
            e_input_dni = e_form.find_element(by=By.NAME,value='form-login[dni]')
            e_input_clave = e_form.find_element(by=By.NAME,value='form-login[clave]')
            # Completar el forumalario
            e_input_dni.send_keys('34178096')
            e_input_clave.send_keys('facundo0301')
            # Enviar el formulario
            e_boton = e_form.find_element(by=By.NAME,value='botones[botonEnviar]')
            e_boton.click()
            WebDriverWait(self.driver, 10).until(
                EC.invisibility_of_element_located(locator=(By.ID, "form-login"))
            )
            self.driver.implicitly_wait(200)
            print('Usuario logueado, redireccionando...\n')        
            # Ingresar a la sección de reserva
            self.u_logueado = True
        except Exception as e:
            print(f"Ocurrió un error en el logeo: {e}.")
    def ingresar_comedor(self,comedor):
        if not self.u_logueado: 
            print('para elegir un comedor primero hay que loguearse')
            return
        # Abrir la página web
        try:
            self.driver.get(self.url_reservas)
            e_contenedor_comedores = WebDriverWait(driver=self.driver, timeout=2).until(
                EC.presence_of_element_located(locator=(By.CLASS_NAME,'reservar-comedores-contenedor'))
            )
            e_botones_comedores = e_contenedor_comedores.find_elements(by=By.TAG_NAME,value='button')
            n_boton_comedor = comedores_df['comedores'].to_list().index(comedor)
            e_boton_salud = e_botones_comedores[n_boton_comedor]
            # Ingresar al comedor de Salud
            e_boton_salud.click()
            print('Ingreso al comedor',comedor)
            self.comedor = comedor
        except Exception as e:
            print(f"Ocurrió un error al elegir un comedor: {e}.")

    def cambiar_mes(self):
        # Se intenta cambiar de me
        span_mes = self.driver.find_element(by=By.XPATH,value="//span[@class='calendario-mes-control-mes']")
        print('ya se localizó el elemento span del mes en curso')
        mes_actual = span_mes.text
        i_mes_actual = self.meses.index(mes_actual)
        buttons_cambios_mes = self.driver.find_elements(by=By.XPATH,value="//button[@class='calendario-mes-control-boton']")
        button_subir_mes = buttons_cambios_mes[1]
        button_subir_mes.click()
        mes_siguiente = self.meses[i_mes_actual+1]
        wait = WebDriverWait(self.driver, 1000)
        wait.until(EC.text_to_be_present_in_element(locator=(By.XPATH,"//span[@class='calendario-mes-control-mes']"),text_=mes_siguiente))
        print('Cambiamos el calandario al próximo mes')

    def buscar_menu(self,menu,dia,mes):
        if self.comedor is None: 
            print('Para reservar un menú primero hay que elegir un comedor')
            return
        try:
            wait = WebDriverWait(self.driver, 100)
            wait.until(EC.visibility_of_all_elements_located(locator=(By.CLASS_NAME,'reservar-servicio')))
            op_reservas = self.driver.find_elements(by=By.CLASS_NAME,value='reservar-servicio')
            n_op_mer = op_menu_df[self.comedor].to_list().index(menu)
            op_mer = op_reservas[n_op_mer]
            op_mer.click()
            self.driver.implicitly_wait(20)
            print(f'Avancemos en reservar el menú {menu} el día {dia} del mes {mes}')
            if dia < date.today().day: self.cambiar_mes()
            lote_span_n_dia = self.driver.find_elements(by=By.XPATH,value="//span[@data-bind='text: numero']")
            span_n_dia_reserva = lote_span_n_dia[dia-1]
            div_class_calendario_dia = span_n_dia_reserva.find_element(by=By.XPATH,value="../..")
            div_class_calendario_dia_atributo_clase = div_class_calendario_dia.get_attribute('class')
            if 'calendario-dia-vacio' in div_class_calendario_dia_atributo_clase:
                print('Aún no se habilitaron las reservas o ese día el comedor está cerrado\n')
                return
            # Chequear que en ese día se otorguen turnos, caso contrario, calendario-dia-vacio
            div_class_calendario_dia_turno = div_class_calendario_dia.find_element(by=By.CLASS_NAME,value="calendario-dia-turno")
            info_enlazada = div_class_calendario_dia_turno.get_attribute(name='data-bind')
            if not info_enlazada:
                print(f'El menú no está disponible: día no habilitado o ya se hizo la reserva\n')
                return 
            div_class_calendario_dia_turno.click()
            wait = WebDriverWait(self.driver, 10)
            div_swal2_container = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "swal2-container")))
            div_swal2_error = self.driver.find_element(By.CLASS_NAME, "swal2-error")
            div_swal2_error_estilo = div_swal2_error.get_attribute('style')
            if div_swal2_error_estilo != 'display: none;': 
                h2_msg_error = div_swal2_container.find_element(by=By.TAG_NAME,value='h2')
                msg_error = h2_msg_error.text
                print(f'Hay problemas con la reserva del menú: {msg_error}\n')
            else:    
                boton_reservar = self.driver.find_element(By.CSS_SELECTOR, ".swal2-confirm")
                boton_reservar.click()
                wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, "swal2-container")))
                # ticket = self.driver.find_element(by=By.CLASS_NAME,value='ticket')
                # print('hay ticket',ticket)
                print(f'Menú reservado\n')
        except Exception as e:
            print(f"Ocurrió un error al elegir un menú: {e}.")
    def buscar_menues(self):
        for _,op_elegida in self.op_eledigas.sort_values(by='fecha').iterrows():
            comedor_elegido = op_elegida['comedor']
            menu_elegido = op_elegida['menu']
            dia_elegido = op_elegida['fecha'].day
            mes_elegido = op_elegida['fecha'].month
            if self.u_logueado: self.ingresar_comedor(comedor=comedor_elegido)
            if self.comedor: self.buscar_menu(menu=menu_elegido,dia=dia_elegido,mes=mes_elegido)
        self.cerrar_navegador()    

In [ ]:
while True:
    hora_actual = time.localtime().tm_hour
    minutos_actual = time.localtime().tm_min
    print(f'Hora actual: {hora_actual} hs con {minutos_actual} minutos. Avancemos en reservar \U0001F4AA!!!\n')
    reserva = Reserva(u_dni='34178096',u_clave='facundo0301',op_eledigas=menues_elegidos_df)
    reserva.loguearse()
    reserva.buscar_menues()
    una_hora = 60 * 10
    sleep(una_hora)

Hora actual: 20 hs con 55 minutos. Avancemos en reservar 💪!!!

Formulario de logeo recibido
Usuario logueado, redireccionando...

Ingreso al comedor salud
Avancemos en reservar el menú desayuno (7:45 - 10.30) el día 2 del mes 6
ya se localizó el elemento span del mes en curso
Cambiamos el calandario al próximo mes
Aún no se habilitaron las reservas o ese día el comedor está cerrado

Ingreso al comedor salud
Avancemos en reservar el menú merienda (16 - 19) el día 2 del mes 6
ya se localizó el elemento span del mes en curso
Cambiamos el calandario al próximo mes
Aún no se habilitaron las reservas o ese día el comedor está cerrado

Ingreso al comedor salud
Avancemos en reservar el menú desayuno (7:45 - 10.30) el día 3 del mes 6
ya se localizó el elemento span del mes en curso
Cambiamos el calandario al próximo mes
Aún no se habilitaron las reservas o ese día el comedor está cerrado

Ingreso al comedor salud
Avancemos en reservar el menú merienda (16 - 19) el día 3 del mes 6
ya se localizó